In [3]:
!pip install xgboost

     |████████████████████████████████| 200.3 MB 27 kB/s  eta 0:00:01    |███                             | 19.0 MB 8.6 MB/s eta 0:00:21     |████                            | 25.1 MB 4.9 MB/s eta 0:00:36     |█████████▍                      | 59.0 MB 455 kB/s eta 0:05:11     |███████████▍                    | 71.4 MB 1.9 MB/s eta 0:01:09     |███████████████████████▍        | 146.3 MB 2.4 MB/s eta 0:00:23     |████████████████████████▏       | 151.5 MB 1.4 MB/s eta 0:00:35     |█████████████████████████████▋  | 185.6 MB 2.1 MB/s eta 0:00:07     |██████████████████████████████▋ | 191.7 MB 2.0 MB/s eta 0:00:05


In [4]:
import pandas as pd
import json
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score
from collections import defaultdict
from pathlib import Path
import os
import itertools 
import numpy as np
import random
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from sklearn.svm import SVC
from sklearn.feature_selection import VarianceThreshold
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, StratifiedKFold
from xgboost import XGBClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.decomposition import PCA
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings("ignore")

In [5]:
seed =1380
np.random.seed(seed)
random.seed(seed) 
os.environ['PYTHONHASHSEED']=str(seed)

In [6]:
fmri_dataset = pd.read_csv('fmri_cobre_initial.csv')
morp_dataset = pd.read_csv('morphometry_initial.csv')

# Define unique IDs for each file

In [5]:
# Read the CSV files
file1 = pd.read_csv('fmri_cobre_initial.csv')
file2 = pd.read_csv('morphometry_initial.csv')

# Extract the ID column from each file
id_col_file1 = file1['ID']
id_col_file2 = file2['ID']

# Find the IDs that are different between the two files
ids_different_file1 = id_col_file1[~id_col_file1.isin(id_col_file2)]
ids_different_file2 = id_col_file2[~id_col_file2.isin(id_col_file1)]

# Print the IDs that are different between the two files
print("IDs in file1.csv that are different from file2.csv:")
print(ids_different_file1)

print("\nIDs in file2.csv that are different from file1.csv:")
print(ids_different_file2)

IDs in file1.csv that are different from file2.csv:
22     A00013363
40     A00016720
43     A00017294
55     A00020805
69     A00022592
84     A00023246
97     A00024372
113    A00027487
124    A00029226
141    A00037224
Name: ID, dtype: object

IDs in file2.csv that are different from file1.csv:
5      A00028402
16     A00038172
19     A00019750
26     A00028189
38     A00035859
79     A00014175
82     A00000300
87     A00020416
88     A00020416
91     A00027755
92     A00027755
98     A00031271
106    A00035836
113    A00027969
Name: ID, dtype: object


# Drop duplicates from morphometry

In [46]:
# Read the 'morphometry_common.csv' file
morphometry_common = pd.read_csv('morphometry_common.csv')

# Drop duplicates based on the 'ID' column
morphometry_common.drop_duplicates(subset='ID', inplace=True)

# Save the updated DataFrame to 'morphometry_common.csv'
morphometry_common.to_csv('morphometry_common.csv', index=False)

print("Duplicates dropped from the 'morphometry_common.csv' file and saved.")

Duplicates dropped from the 'morphometry_common.csv' file and saved.


# Define datsets for each modality

In [58]:
fmri_dataset = pd.read_csv('fmri_common_cobre.csv')
morp_dataset = pd.read_csv('morphometry_common.csv')

# Concatennate datasets

In [52]:
# Path to the CSV files
file1_path = 'morphometry_common.csv'
file2_path = 'fmri_common_cobre.csv'

# Read the CSV files
df1 = pd.read_csv(file1_path)
df2 = pd.read_csv(file2_path)

# Concatenate the dataframes based on the ID column
concatenated_df = pd.merge(df1, df2, on='ID')

# Save the concatenated dataframe to a new CSV file
output_path = 'fmri_and_morph_3.07.csv'
concatenated_df.to_csv(output_path, index=False)

print("CSV files concatenated based on the ID column and saved as 'concatenated.csv'.")

CSV files concatenated based on the ID column and saved as 'concatenated.csv'.


# Label target in morphometry

In [55]:
# Read the CSV file
df = pd.read_csv('morphometry_common.csv')

# Filter rows where the 'target' column is not equal to 'Schizoaffective'
df = df[df['target'] != 'Schizoaffective']

df['target'].replace({'Schizophrenia_Strict': 0, 'No_Known_Disorder': 1}, inplace=True)

# Save the modified dataframe to a new CSV file
output_path = 'morphometry_common.csv'
df.to_csv(output_path, index=False)

# Look at shape and target values

In [60]:
morp_dataset.shape

(142, 931)

In [59]:
morp_dataset.target.value_counts()

1    76
0    66
Name: target, dtype: int64

In [52]:
fmri_dataset.shape

(142, 6672)

In [53]:
fmri_dataset.target.value_counts()

0    76
1    66
Name: target, dtype: int64

In [7]:
morh_and_fmri = pd.read_csv('fmri_and_morph_3.07.csv')

In [54]:
morh_and_fmri.target.value_counts()

0    76
1    66
Name: target, dtype: int64

# Code to doublecheck if Schizoaffective is drop from target

In [9]:
# # Read the datasets
# fmri_dataset = pd.read_csv('fmri_common_cobre.csv')
# morp_dataset = pd.read_csv('morphometry_common.csv')
# morh_and_fmri = pd.read_csv('fmri_and_morph_3.07.csv')

# # Drop rows with "Schizoaffective" in the target column
# fmri_dataset = fmri_dataset[fmri_dataset['target'] != 'Schizoaffective']
# morp_dataset = morp_dataset[morp_dataset['target'] != 'Schizoaffective']
# morh_and_fmri = morh_and_fmri[morh_and_fmri['target'] != 'Schizoaffective']

# # Save the modified datasets back to the original files
# fmri_dataset.to_csv('fmri_cobre_initial.csv', index=False)
# morp_dataset.to_csv('morphometry_initial.csv', index=False)
# morh_and_fmri.to_csv('fmri_and_morph_3.07.csv', index=False)

In [10]:
# fmri_dataset = pd.read_csv('fmri_common_cobre.csv')
# morp_dataset = pd.read_csv('morphometry_common.csv')
# morh_and_fmri = pd.read_csv('fmri_and_morph_3.07.csv')

# Early fusion for fMRI dataset

In [61]:
target =fmri_dataset.target
data =fmri_dataset.drop(columns=['target', 'ID'])

In [63]:
# Define the pipeline for feature selection and model training
pipeline_rf = Pipeline([
    ("scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=92)),
    ("model", RandomForestClassifier(random_state=seed))
])

pipeline_svc = Pipeline([
    ("scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=92)),
    ("model", SVC(random_state=seed))
])

pipeline_xgb = Pipeline([
    ("scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=92)),
    ("model", XGBClassifier(random_state=seed))
])

In [64]:
# Define the parameter grids for each model
param_grid_rf = {
    'model__n_estimators': [10, 50, 100],
    'model__max_depth': [3, 5, 7, 9],
    'model__random_state': [1380]
}

param_grid_svc = {
    'model__C': [0.01, 0.1, 1],
    'model__class_weight': ['balanced', None],
    'model__kernel': ['linear', 'rbf'],
    'model__probability': [True],
    'model__random_state': [1380]
}

param_grid_xgb = {
    'model__learning_rate': [0.1],
    'model__max_depth': [6, 7, 3, 10],
    'model__scale_pos_weight': [1],
    'model__subsample': [1, 0.5],
    'model__alpha': [0],
    'model__random_state': [1380],
    'model__use_label_encoder': [False],
    'model__objective': ['binary:logistic'],
    
}

In [65]:
# Declare the inner and outer cross-validation strategies
inner_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

# Models to evaluate
models = {
    'Random Forest': (pipeline_rf, param_grid_rf),
    'SVC': (pipeline_svc, param_grid_svc),
    'XGBoost': (pipeline_xgb, param_grid_xgb)
}

# Perform nested cross-validation for each model
for model_name, (pipeline, param_grid) in models.items():
    # Inner cross-validation for parameter search
    model = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=inner_cv, n_jobs=1, scoring='f1_macro')

    # Outer cross-validation to compute the testing score
    test_score = cross_val_score(model, data, target, cv=outer_cv, n_jobs=1, scoring='f1_macro')

    print(f"{model_name}: Mean score using nested cross-validation is: {test_score.mean():.3f} ± {test_score.std():.3f}")

Random Forest: Mean score using nested cross-validation is: 0.636 ± 0.117
SVC: Mean score using nested cross-validation is: 0.659 ± 0.117
XGBoost: Mean score using nested cross-validation is: 0.656 ± 0.105


# fMRI dataset. Scores estimation without feature extraction

In [17]:
target =fmri_dataset.target
data =fmri_dataset.drop(columns=['target', 'ID'])

In [18]:
# Define the pipeline for feature selection and model training
pipeline_lr = Pipeline([
    ("scaler", StandardScaler()),
    ("model", LogisticRegression(random_state=seed))
])

pipeline_rf = Pipeline([
    ("scaler", StandardScaler()),
    ("model", RandomForestClassifier(random_state=seed))
])

pipeline_svc = Pipeline([
    ("scaler", StandardScaler()),
    ("model", SVC(random_state=seed))
])

pipeline_xgb = Pipeline([
    ("scaler", StandardScaler()),
    ("model", XGBClassifier(random_state=seed))
])

In [19]:
# Define the parameter grids for each model
param_grid_lr = {
    'model__C': [0.01, 0.1, 1],
    'model__solver': ['liblinear'],
    'model__penalty': ['l1', 'l2'],
    'model__random_state': [1380]
}

param_grid_rf = {
    'model__n_estimators': [10, 50, 100],
    'model__max_depth': [3, 5, 7, 9],
    'model__random_state': [1380]
}

param_grid_svc = {
    'model__C': [0.01, 0.1, 1],
    'model__class_weight': ['balanced', None],
    'model__kernel': ['linear', 'rbf'],
    'model__probability': [True],
    'model__random_state': [1380]
}

param_grid_xgb = {
    'model__learning_rate': [0.1],
    'model__max_depth': [6, 7, 3, 10],
    'model__scale_pos_weight': [1],
    'model__subsample': [1, 0.5],
    'model__alpha': [0],
    'model__random_state': [1380],
    'model__use_label_encoder': [False],
    'model__objective': ['binary:logistic'],
    
}

In [21]:
# Declare the inner and outer cross-validation strategies
inner_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

# Models to evaluate
models = {
    'Logistic Regression': (pipeline_lr, param_grid_lr),
    'Random Forest': (pipeline_rf, param_grid_rf),
    'SVC': (pipeline_svc, param_grid_svc),
    'XGBoost': (pipeline_xgb, param_grid_xgb)
}

# Perform nested cross-validation for each model
for model_name, (pipeline, param_grid) in models.items():
    # Inner cross-validation for parameter search
    model = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=inner_cv, n_jobs=-1, scoring='f1_macro')

    # Outer cross-validation to compute the testing score
    test_score = cross_val_score(model, data, target, cv=outer_cv, n_jobs=-1, scoring='f1_macro')

    print(f"{model_name}: Mean score using nested cross-validation is: {test_score.mean():.3f} ± {test_score.std():.3f}")

Logistic Regression: Mean score using nested cross-validation is: 0.721 ± 0.133


/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:101: FutureWarning: Attribute _pairwise was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)


Random Forest: Mean score using nested cross-validation is: 0.662 ± 0.129
SVC: Mean score using nested cross-validation is: 0.728 ± 0.128


/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `us

XGBoost: Mean score using nested cross-validation is: 0.698 ± 0.105


# Early fusion for fMRI+morphometry dataset

In [71]:
target =morh_and_fmri.target
data =morh_and_fmri.drop(columns=['target', 'ID'])

In [72]:
# Define the pipeline for feature selection and model training
pipeline_rf = Pipeline([
    ("scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=92)),
    ("model", RandomForestClassifier(random_state=seed))
])

pipeline_svc = Pipeline([
    ("scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=92)),
    ("model", SVC(random_state=seed))
])

pipeline_xgb = Pipeline([
    ("scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=92)),
    ("model", XGBClassifier(random_state=seed))
])

In [73]:
# Define the parameter grids for each model
param_grid_rf = {
    'model__n_estimators': [10, 50, 100],
    'model__max_depth': [3, 5, 7, 9],
    'model__random_state': [1380]
}

param_grid_svc = {
    'model__C': [0.01, 0.1, 1],
    'model__class_weight': ['balanced', None],
    'model__kernel': ['linear', 'rbf'],
    'model__probability': [True],
    'model__random_state': [1380]
}

param_grid_xgb = {
    'model__learning_rate': [0.1],
    'model__max_depth': [6, 7, 3, 10],
    'model__scale_pos_weight': [1],
    'model__subsample': [1, 0.5],
    'model__alpha': [0],
    'model__random_state': [1380],
    'model__use_label_encoder': [False],
    'model__objective': ['binary:logistic'],
    
}

In [74]:
# Declare the inner and outer cross-validation strategies
inner_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

# Models to evaluate
models = {
    'Random Forest': (pipeline_rf, param_grid_rf),
    'SVC': (pipeline_svc, param_grid_svc),
    'XGBoost': (pipeline_xgb, param_grid_xgb)
}

# Perform nested cross-validation for each model
for model_name, (pipeline, param_grid) in models.items():
    # Inner cross-validation for parameter search
    model = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=inner_cv, n_jobs=1, scoring='f1_macro')

    # Outer cross-validation to compute the testing score
    test_score = cross_val_score(model, data, target, cv=outer_cv, n_jobs=1, scoring='f1_macro')

    print(f"{model_name}: Mean score using nested cross-validation is: {test_score.mean():.3f} ± {test_score.std():.3f}")

Random Forest: Mean score using nested cross-validation is: 0.675 ± 0.134
SVC: Mean score using nested cross-validation is: 0.652 ± 0.144
XGBoost: Mean score using nested cross-validation is: 0.719 ± 0.092


# fMRI+morphometry dataset. Scores estimation without feature extraction

In [23]:
target =morh_and_fmri.target
data =morh_and_fmri.drop(columns=['target', 'ID'])

In [24]:
# Define the pipeline for feature selection and model training
pipeline_lr = Pipeline([
    ("scaler", StandardScaler()),
    ("model", LogisticRegression(random_state=seed))
])

pipeline_rf = Pipeline([
    ("scaler", StandardScaler()),
    ("model", RandomForestClassifier(random_state=seed))
])

pipeline_svc = Pipeline([
    ("scaler", StandardScaler()),
    ("model", SVC(random_state=seed))
])

pipeline_xgb = Pipeline([
    ("scaler", StandardScaler()),
    ("model", XGBClassifier(random_state=seed))
])

In [25]:
# Define the parameter grids for each model
param_grid_lr = {
    'model__C': [0.01, 0.1, 1],
    'model__solver': ['liblinear'],
    'model__penalty': ['l1', 'l2'],
    'model__random_state': [1380]
}

param_grid_rf = {
    'model__n_estimators': [10, 50, 100],
    'model__max_depth': [3, 5, 7, 9],
    'model__random_state': [1380]
}

param_grid_svc = {
    'model__C': [0.01, 0.1, 1],
    'model__class_weight': ['balanced', None],
    'model__kernel': ['linear', 'rbf'],
    'model__probability': [True],
    'model__random_state': [1380]
}

param_grid_xgb = {
    'model__learning_rate': [0.1],
    'model__max_depth': [6, 7, 3, 10],
    'model__scale_pos_weight': [1],
    'model__subsample': [1, 0.5],
    'model__alpha': [0],
    'model__random_state': [1380],
    'model__use_label_encoder': [False],
    'model__objective': ['binary:logistic'],
    
}

In [26]:
# Declare the inner and outer cross-validation strategies
inner_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

# Models to evaluate
models = {
    'Logistic Regression': (pipeline_lr, param_grid_lr),
    'Random Forest': (pipeline_rf, param_grid_rf),
    'SVC': (pipeline_svc, param_grid_svc),
    'XGBoost': (pipeline_xgb, param_grid_xgb)
}

# Perform nested cross-validation for each model
for model_name, (pipeline, param_grid) in models.items():
    # Inner cross-validation for parameter search
    model = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=inner_cv, n_jobs=-1, scoring='f1_macro')

    # Outer cross-validation to compute the testing score
    test_score = cross_val_score(model, data, target, cv=outer_cv, n_jobs=-1, scoring='f1_macro')

    print(f"{model_name}: Mean score using nested cross-validation is: {test_score.mean():.3f} ± {test_score.std():.3f}")

/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:101: FutureWarning: Attribute _pairwise was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)


Logistic Regression: Mean score using nested cross-validation is: 0.682 ± 0.117


/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:101: FutureWarning: Attribute _pairwise was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:101: FutureWarning: Attribute _pairwise was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)


Random Forest: Mean score using nested cross-validation is: 0.613 ± 0.156


/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:101: FutureWarning: Attribute _pairwise was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:101: FutureWarning: Attribute _pairwise was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:101: FutureWarning: Attribute _pairwise was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:101: FutureWarning: Attribute _pairwise was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)


SVC: Mean score using nested cross-validation is: 0.687 ± 0.110


/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:101: FutureWarning: Attribute _pairwise was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:101: FutureWarning: Attribute _pairwise was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecate

XGBoost: Mean score using nested cross-validation is: 0.749 ± 0.152


# Early fusion for morphometry dataset

In [42]:
target =morp_dataset.target
data =morp_dataset.drop(columns=['target', 'ID', 'file', 'Sex'])

In [43]:
# Define the pipeline for feature selection and model training
pipeline_rf = Pipeline([
    ("scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=92)),
    ("model", RandomForestClassifier(random_state=seed))
])

pipeline_svc = Pipeline([
    ("scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=92)),
    ("model", SVC(random_state=seed))
])

pipeline_xgb = Pipeline([
    ("scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=92)),
    ("model", XGBClassifier(random_state=seed))
])

In [44]:
# Define the parameter grids for each model
param_grid_rf = {
    'model__n_estimators': [10, 50, 100],
    'model__max_depth': [3, 5, 7, 9],
    'model__random_state': [1380]
}

param_grid_svc = {
    'model__C': [0.01, 0.1, 1],
    'model__class_weight': ['balanced', None],
    'model__kernel': ['linear', 'rbf'],
    'model__probability': [True],
    'model__random_state': [1380]
}

param_grid_xgb = {
    'model__learning_rate': [0.1],
    'model__max_depth': [6, 7, 3, 10],
    'model__scale_pos_weight': [1],
    'model__subsample': [1, 0.5],
    'model__alpha': [0],
    'model__random_state': [1380],
    'model__use_label_encoder': [False],
    'model__objective': ['binary:logistic'],
    
}

In [45]:
# Declare the inner and outer cross-validation strategies
inner_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

# Models to evaluate
models = {
    'Random Forest': (pipeline_rf, param_grid_rf),
    'SVC': (pipeline_svc, param_grid_svc),
    'XGBoost': (pipeline_xgb, param_grid_xgb)
}

# Perform nested cross-validation for each model
for model_name, (pipeline, param_grid) in models.items():
    # Inner cross-validation for parameter search
    model = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=inner_cv, n_jobs=1, scoring='f1_macro')

    # Outer cross-validation to compute the testing score
    test_score = cross_val_score(model, data, target, cv=outer_cv, n_jobs=1, scoring='f1_macro')

    print(f"{model_name}: Mean score using nested cross-validation is: {test_score.mean():.3f} ± {test_score.std():.3f}")

Random Forest: Mean score using nested cross-validation is: 0.567 ± 0.114
SVC: Mean score using nested cross-validation is: 0.565 ± 0.119
XGBoost: Mean score using nested cross-validation is: 0.565 ± 0.093


# Morphometry dataset. Scores estimation without feature extraction

In [20]:
target =morp_dataset.target
data =morp_dataset.drop(columns=['target', 'ID', 'file', 'Sex'])

In [21]:
# Define the pipeline for feature selection and model training
pipeline_lr = Pipeline([
    ("scaler", StandardScaler()),
    ("model", LogisticRegression(random_state=seed))
])

pipeline_rf = Pipeline([
    ("scaler", StandardScaler()),
    ("model", RandomForestClassifier(random_state=seed))
])

pipeline_svc = Pipeline([
    ("scaler", StandardScaler()),
    ("model", SVC(random_state=seed))
])

pipeline_xgb = Pipeline([
    ("scaler", StandardScaler()),
    ("model", XGBClassifier(random_state=seed))
])

In [22]:
# Define the parameter grids for each model
param_grid_lr = {
    'model__C': [0.01, 0.1, 1],
    'model__solver': ['liblinear'],
    'model__penalty': ['l1', 'l2'],
    'model__random_state': [1380]
}

param_grid_rf = {
    'model__n_estimators': [10, 50, 100],
    'model__max_depth': [3, 5, 7, 9],
    'model__random_state': [1380]
}

param_grid_svc = {
    'model__C': [0.01, 0.1, 1],
    'model__class_weight': ['balanced', None],
    'model__kernel': ['linear', 'rbf'],
    'model__probability': [True],
    'model__random_state': [1380]
}

param_grid_xgb = {
    'model__learning_rate': [0.1],
    'model__max_depth': [6, 7, 3, 10],
    'model__scale_pos_weight': [1],
    'model__subsample': [1, 0.5],
    'model__alpha': [0],
    'model__random_state': [1380],
    'model__use_label_encoder': [False],
    'model__objective': ['binary:logistic'],
    
}

In [23]:
# Declare the inner and outer cross-validation strategies
inner_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

# Models to evaluate
models = {
    'Logistic Regression': (pipeline_lr, param_grid_lr),
    'Random Forest': (pipeline_rf, param_grid_rf),
    'SVC': (pipeline_svc, param_grid_svc),
    'XGBoost': (pipeline_xgb, param_grid_xgb)
}

# Perform nested cross-validation for each model
for model_name, (pipeline, param_grid) in models.items():
    # Inner cross-validation for parameter search
    model = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=inner_cv, n_jobs=-1, scoring='f1_macro')

    # Outer cross-validation to compute the testing score
    test_score = cross_val_score(model, data, target, cv=outer_cv, n_jobs=-1, scoring='f1_macro')

    print(f"{model_name}: Mean score using nested cross-validation is: {test_score.mean():.3f} ± {test_score.std():.3f}")

Logistic Regression: Mean score using nested cross-validation is: 0.662 ± 0.095
Random Forest: Mean score using nested cross-validation is: 0.581 ± 0.117
SVC: Mean score using nested cross-validation is: 0.609 ± 0.114


/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:101: FutureWarning: Attribute _pairwise was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/conda/lib/python3.9/site-pac

XGBoost: Mean score using nested cross-validation is: 0.631 ± 0.083


# Late fusion. ML models without feature selection

### Logistic Regression

In [8]:
# custom transformer for sklearn pipeline
class ColumnExtractor(TransformerMixin, BaseEstimator):
    def __init__(self, cols):
        self.cols = cols

    def transform(self, X):
        col_list = []
        for c in self.cols:
            col_list.append(X.iloc[:, c:c+1])
        return np.concatenate(col_list, axis=1)

    def fit(self, X, y=None):
        return self

In [92]:
   # fit clf1 with df1
pipe1 = Pipeline([
    ('col_extract', ColumnExtractor( cols=range(0,927) )),
    ( "scaler", StandardScaler()),
    ('clf', LogisticRegression(random_state=seed))
    ])


pipe2 = Pipeline([
    ('col_extract', ColumnExtractor( cols=range(927,7598) )),
    ( "scaler", StandardScaler()),
    ('clf', LogisticRegression(random_state=seed))
    ])


eclf = VotingClassifier(estimators=[('df1-clf1', pipe1), ('df2-clf2', pipe2)], voting='soft')

In [96]:
target =morh_and_fmri.target
data_late =morh_and_fmri.drop(columns=['target', 'ID'])

In [97]:
param_grid = [
    {
        'df1-clf1__clf__penalty': ['l1', 'l2'],
        'df1-clf1__clf__C': [0.1, 1, 10],
        'df1-clf1__clf__fit_intercept': [True, False],
        'df1-clf1__clf__solver': ['liblinear'],
        'df1-clf1__clf__random_state': [1380]
    },
    {
        'df2-clf2__clf__penalty': ['l1', 'l2'],
        'df2-clf2__clf__C': [0.1, 1, 10],
        'df2-clf2__clf__fit_intercept': [True, False],
        'df2-clf2__clf__solver': ['liblinear'],
        'df2-clf2__clf__random_state': [1380]
    }
]

In [98]:
from sklearn.model_selection import cross_val_score, KFold

# Declare the inner and outer cross-validation strategies
inner_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1380)
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1380)

# Inner cross-validation for parameter search
model = GridSearchCV(
    estimator=eclf, param_grid=param_grid, cv=inner_cv, n_jobs=-1
)

# Outer cross-validation to compute the testing score
test_score = cross_val_score(model, data_late, target, cv=outer_cv, n_jobs=1,scoring='f1_macro')
print(f"The mean score using nested cross-validation is: "
      f"{test_score.mean():.3f} ± {test_score.std():.3f}")

The mean score using nested cross-validation is: 0.740 ± 0.129


### Random forest

In [99]:
   # fit clf1 with df1
pipe1 = Pipeline([
    ('col_extract', ColumnExtractor( cols=range(0,927) )),
    ( "scaler", StandardScaler()),
    ('clf', RandomForestClassifier(random_state=seed))
    ])


pipe2 = Pipeline([
    ('col_extract', ColumnExtractor( cols=range(927,7598) )),
    ( "scaler", StandardScaler()),
    ('clf', RandomForestClassifier(random_state=seed))
    ])


eclf = VotingClassifier(estimators=[('df1-clf1', pipe1), ('df2-clf2', pipe2)], voting='soft')

In [ ]:
target =morh_and_fmri.target
data_late =morh_and_fmri.drop(columns=['target', 'ID'])

In [100]:
param_grid = [
    {
        'df1-clf1__clf__n_estimators': [10, 50, 100],
        'df1-clf1__clf__max_depth': [3, 5, 7, 9],
        'df1-clf1__clf__random_state': [1380]
    },
    {
        'df2-clf2__clf__n_estimators': [10, 50, 100],
        'df2-clf2__clf__max_depth': [3, 5, 7, 9],
        'df2-clf2__clf__random_state': [1380]
    }
]

In [101]:
from sklearn.model_selection import cross_val_score, KFold

# Declare the inner and outer cross-validation strategies
inner_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1380)
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1380)

# Inner cross-validation for parameter search
model = GridSearchCV(
    estimator=eclf, param_grid=param_grid, cv=inner_cv, n_jobs=-1
)

# Outer cross-validation to compute the testing score
test_score = cross_val_score(model, data_late, target, cv=outer_cv, n_jobs=1,scoring='f1_macro')
print(f"The mean score using nested cross-validation is: "
      f"{test_score.mean():.3f} ± {test_score.std():.3f}")

The mean score using nested cross-validation is: 0.652 ± 0.138


### Random Forest+feature extraction from Logistic Regression

In [45]:
# custom transformer for sklearn pipeline
class ColumnExtractor(TransformerMixin, BaseEstimator):
    def __init__(self, cols):
        self.cols = cols

    def transform(self, X):
        col_list = []
        for c in self.cols:
            col_list.append(X.iloc[:, c:c+1])
        return np.concatenate(col_list, axis=1)

    def fit(self, X, y=None):
        return self

In [46]:
   # fit clf1 with df1
pipe1 = Pipeline([
    ('col_extract', ColumnExtractor( cols=range(0,927) )), 
    ( "scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=92)),
    ('clf', RandomForestClassifier(random_state=seed))
    ])


pipe2 = Pipeline([
    ('col_extract', ColumnExtractor( cols=range(927,7598) )),
    ( "scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=92)),
    ('clf', RandomForestClassifier(random_state=seed))
    ])


eclf = VotingClassifier(estimators=[('df1-clf1', pipe1), ('df2-clf2', pipe2)], voting='soft')


In [47]:
target =morh_and_fmri.target
data_late =morh_and_fmri.drop(columns=['target', 'ID'])

In [48]:
param_grid = [
    {
        'df1-clf1__clf__n_estimators': [10, 50, 100],
        'df1-clf1__clf__max_depth': [3, 5, 7, 9],
        'df1-clf1__clf__random_state': [1380]
    },
    {
        'df2-clf2__clf__n_estimators': [10, 50, 100],
        'df2-clf2__clf__max_depth': [3, 5, 7, 9],
        'df2-clf2__clf__random_state': [1380]
    }
]

In [49]:
from sklearn.model_selection import cross_val_score, KFold

# Declare the inner and outer cross-validation strategies
inner_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1380)
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1380)

# Inner cross-validation for parameter search
model = GridSearchCV(
    estimator=eclf, param_grid=param_grid, cv=inner_cv, n_jobs=-1
)

# Outer cross-validation to compute the testing score
test_score = cross_val_score(model, data_late, target, cv=outer_cv, n_jobs=-1,scoring='f1_macro')
print(f"The mean score using nested cross-validation is: "
      f"{test_score.mean():.3f} ± {test_score.std():.3f}")

The mean score using nested cross-validation is: 0.734 ± 0.123


### SVC

In [31]:
# custom transformer for sklearn pipeline
class ColumnExtractor(TransformerMixin, BaseEstimator):
    def __init__(self, cols):
        self.cols = cols

    def transform(self, X):
        col_list = []
        for c in self.cols:
            col_list.append(X.iloc[:, c:c+1])
        return np.concatenate(col_list, axis=1)

    def fit(self, X, y=None):
        return self

In [32]:
   # fit clf1 with df1
pipe1 = Pipeline([
    ('col_extract', ColumnExtractor( cols=range(0,927) )), 
    ( "scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=92)),
    ('clf', SVC(random_state=seed, probability=True))
    ])


pipe2 = Pipeline([
    ('col_extract', ColumnExtractor( cols=range(927,7598) )),
    ( "scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=92)),
    ('clf', SVC(random_state=seed,probability=True))
    ])


eclf = VotingClassifier(estimators=[('df1-clf1', pipe1), ('df2-clf2', pipe2)], voting='soft')


In [33]:
target =morh_and_fmri.target
data_late =morh_and_fmri.drop(columns=['target', 'ID'])

In [34]:
target

0      0
1      1
2      1
3      1
4      1
      ..
137    0
138    0
139    0
140    1
141    0
Name: target, Length: 142, dtype: int64

In [35]:
param_grid = [
    {
        'df1-clf1__clf__C': [0.01, 0.1, 1],
        'df1-clf1__clf__class_weight': ['balanced', None],
        'df1-clf1__clf__kernel': ['linear', 'rbf'],
        'df1-clf1__clf__probability': [True],
        'df1-clf1__clf__random_state': [seed]
    },
    {
        'df2-clf2__clf__C': [0.01, 0.1, 1],
        'df2-clf2__clf__class_weight': ['balanced', None],
        'df2-clf2__clf__kernel': ['linear', 'rbf'],
        'df2-clf2__clf__probability': [True],
        'df2-clf2__clf__random_state': [seed]
    }
]

In [36]:
from sklearn.model_selection import cross_val_score, KFold

# Declare the inner and outer cross-validation strategies
inner_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1380)
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1380)

# Inner cross-validation for parameter search
model = GridSearchCV(
    estimator=eclf, param_grid=param_grid, cv=inner_cv, n_jobs=-1
)

# Outer cross-validation to compute the testing score
test_score = cross_val_score(model, data_late, target, cv=outer_cv, n_jobs=-1,scoring='f1_macro')
print(f"The mean score using nested cross-validation is: "
      f"{test_score.mean():.3f} ± {test_score.std():.3f}")

The mean score using nested cross-validation is: 0.732 ± 0.091


### SVC+feature extraction from Logistic Regression

In [8]:
# custom transformer for sklearn pipeline
class ColumnExtractor(TransformerMixin, BaseEstimator):
    def __init__(self, cols):
        self.cols = cols

    def transform(self, X):
        col_list = []
        for c in self.cols:
            col_list.append(X.iloc[:, c:c+1])
        return np.concatenate(col_list, axis=1)

    def fit(self, X, y=None):
        return self

In [9]:
   # fit clf1 with df1
pipe1 = Pipeline([
    ('col_extract', ColumnExtractor( cols=range(0,927) )), 
    ( "scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=92)),
    ('clf', SVC(random_state=seed,probability=True))
    ])


pipe2 = Pipeline([
    ('col_extract', ColumnExtractor( cols=range(927,7598) )),
    ( "scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=92)),
    ('clf', SVC(random_state=seed,probability=True))
    ])


eclf = VotingClassifier(estimators=[('df1-clf1', pipe1), ('df2-clf2', pipe2)], voting='soft')


In [10]:
target =morh_and_fmri.target
data_late =morh_and_fmri.drop(columns=['target', 'ID'])

In [11]:
param_grid = [
    {
        'df1-clf1__clf__C': [0.01, 0.1, 1],
        'df1-clf1__clf__class_weight': ['balanced', None],
        'df1-clf1__clf__kernel': ['linear', 'rbf'],
        'df1-clf1__clf__probability': [True],
        'df1-clf1__clf__random_state': [seed]
    },
    {
        'df2-clf2__clf__C': [0.01, 0.1, 1],
        'df2-clf2__clf__class_weight': ['balanced', None],
        'df2-clf2__clf__kernel': ['linear', 'rbf'],
        'df2-clf2__clf__probability': [True],
        'df2-clf2__clf__random_state': [seed]
    }
]

In [12]:
from sklearn.model_selection import cross_val_score, KFold

# Declare the inner and outer cross-validation strategies
inner_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1380)
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1380)

# Inner cross-validation for parameter search
model = GridSearchCV(
    estimator=eclf, param_grid=param_grid, cv=inner_cv, n_jobs=-1
)

# Outer cross-validation to compute the testing score
test_score = cross_val_score(model, data_late, target, cv=outer_cv, n_jobs=-1,scoring='f1_macro')
print(f"The mean score using nested cross-validation is: "
      f"{test_score.mean():.3f} ± {test_score.std():.3f}")

The mean score using nested cross-validation is: 0.732 ± 0.091


### XGBoost

In [13]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from xgboost import XGBClassifier
import numpy as np

# Load the data
target = morh_and_fmri.target
data_late = morh_and_fmri.drop(columns=['target', 'ID'])

# Custom transformer for sklearn pipeline
class ColumnExtractor(TransformerMixin, BaseEstimator):
    def __init__(self, cols):
        self.cols = cols

    def transform(self, X):
        col_list = []
        for c in self.cols:
            col_list.append(X.iloc[:, c:c+1])
        return np.concatenate(col_list, axis=1)

    def fit(self, X, y=None):
        return self


# Define the pipeline
pipe1 = Pipeline([
    ('col_extract', ColumnExtractor(cols=range(0, 927))),
    ("scaler", StandardScaler()),
    ('clf', XGBClassifier(random_state=seed, use_label_encoder=False, objective='binary:logistic'))
])

pipe2 = Pipeline([
    ('col_extract', ColumnExtractor(cols=range(927, 7598))),
    ("scaler", StandardScaler()),
    ('clf', XGBClassifier(random_state=seed, use_label_encoder=False, objective='binary:logistic'))
])


# Define the parameter grid
param_grid = [
    {
        'clf__learning_rate': [0.1],
        'clf__max_depth': [6, 7, 3, 10],
        'clf__scale_pos_weight': [1],
        'clf__subsample': [1, 0.5],
        'clf__alpha': [0],
        'clf__random_state': [1380],
        'clf__use_label_encoder': [False],
        'clf__objective': ['binary:logistic']
    }
]

# Declare the inner and outer cross-validation strategies
inner_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1380)
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1380)

# Inner cross-validation for parameter search
model = GridSearchCV(estimator=pipe1, param_grid=param_grid, cv=inner_cv, n_jobs=-1)

# Outer cross-validation to compute the testing score
test_score = cross_val_score(model, data_late, target, cv=outer_cv, n_jobs=1, scoring='f1_macro')
print(f"The mean score using nested cross-validation is: {test_score.mean():.3f} ± {test_score.std():.3f}")

The mean score using nested cross-validation is: 0.631 ± 0.083


### XGBoost+feature extraction from Logistic Regression

In [14]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from xgboost import XGBClassifier
import numpy as np

# Load the data
target = morh_and_fmri.target
data_late = morh_and_fmri.drop(columns=['target', 'ID'])

# Custom transformer for sklearn pipeline
class ColumnExtractor(TransformerMixin, BaseEstimator):
    def __init__(self, cols):
        self.cols = cols

    def transform(self, X):
        col_list = []
        for c in self.cols:
            col_list.append(X.iloc[:, c:c+1])
        return np.concatenate(col_list, axis=1)

    def fit(self, X, y=None):
        return self


# Define the pipeline
pipe1 = Pipeline([
    ('col_extract', ColumnExtractor(cols=range(0, 927))),
    ("scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=92)),
    ('clf', XGBClassifier(random_state=seed, use_label_encoder=False, objective='binary:logistic'))
])

pipe2 = Pipeline([
    ('col_extract', ColumnExtractor(cols=range(927, 7598))),
    ("scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=92)),
    ('clf', XGBClassifier(random_state=seed, use_label_encoder=False, objective='binary:logistic'))
])


# Define the parameter grid
param_grid = [
    {
        'clf__learning_rate': [0.1],
        'clf__max_depth': [6, 7, 3, 10],
        'clf__scale_pos_weight': [1],
        'clf__subsample': [1, 0.5],
        'clf__alpha': [0],
        'clf__random_state': [1380],
        'clf__use_label_encoder': [False],
        'clf__objective': ['binary:logistic']
    }
]

# Declare the inner and outer cross-validation strategies
inner_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1380)
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1380)

# Inner cross-validation for parameter search
model = GridSearchCV(estimator=pipe1, param_grid=param_grid, cv=inner_cv, n_jobs=-1)

# Outer cross-validation to compute the testing score
test_score = cross_val_score(model, data_late, target, cv=outer_cv, n_jobs=1, scoring='f1_macro')
print(f"The mean score using nested cross-validation is: {test_score.mean():.3f} ± {test_score.std():.3f}")


The mean score using nested cross-validation is: 0.552 ± 0.076
